In [1]:
'''        
    Integrantes do grupo:
        Tamiris Fernandes Tinelli
        Giovani Decico Lucafó
        Alberto Campos Neves
        
    Projeto:
        Jogo interativo com temática de halloween.
        No cenario temos:
            - Um caldeirão que borbulha infinitamente.
            - Assassino que abaixa e levanta o braço usando as setas UP e DOWN do teclado
            - Bruxa que voa de um lado para o outro usando as setas LEFT e RIGHT do teclado
            - Morcego que voa em direcao ao observador e volta usando a tecla B do teclado.
            - Gato que dorme e acorda (abrindo e fechando os olhos) usando a tecla S e W do teclado, respectivamente.
            
'''

'        \n    Integrantes do grupo:\n        Tamiris Fernandes Tinelli\n        Giovani Decico Lucafó\n        Alberto Campos Neves\n        \n    Projeto:\n        Jogo interativo com temática de halloween.\n        No cenario temos:\n            - Um caldeirão que borbulha infinitamente.\n            - Assassino que abaixa e levanta o braço usando as setas UP e DOWN do teclado\n            - Bruxa que voa de um lado para o outro usando as setas LEFT e RIGHT do teclado\n            - Morcego que voa em direcao ao observador e volta usando a tecla B do teclado.\n            - Gato que dorme e acorda (abrindo e fechando os olhos) usando a tecla S e W do teclado, respectivamente.\n            \n'

In [2]:
from window import *
from objects2d import *
import math

In [3]:
#Keyboard and Mouse Events Capture

#WITCH
witch_tx = 0.0
witch_spin = False
witch_theta = 0.0

#CAT
cat_sleeping = False
s_eyes = 1.0

#CAULDRON
t_bubble = 0.0
#BAT
bat_scale = False
s_x = 1.0
s_y = 1.0

#ASSASSIN
arm_rotation = 0.0

aux_tx = 0.0
aux_ty = 0.0


def key_event(window,key,scancode,action,mods):
    global witch_tx, witch_spin
    global cat_sleeping
    global bat_scale
    global arm_rotation
    global aux_tx, aux_ty
    
    #WITCH
    if key == 262: #right
        witch_tx += 0.01
    elif key == 263: #left
        witch_tx -= 0.01
        
    elif key == 32: # space
        witch_spin = True
    
    #CAT   
    elif key == 83: # S
        cat_sleeping = True
    elif key == 87: # W
        cat_sleeping = False
        
    #BAT
    elif key == 66: # B
        bat_scale = True
    
    #ASSASSIN
    elif key == 264: #down
        arm_rotation += 0.05
    elif key == 265: #up
        arm_rotation -= 0.05 
    
    

def mouse_event(window,button,action,mods):
    pass

In [4]:
def getCircleCoordinates(n_vertices, radius, center = (0, 0)):
    '''
    Get coordinates to draw a circle.
    Parameters
    ----------
        n_vertices : int
            Number of vertices that will be used to draw the circle.
        radius : float
            Radius of circle
        center : (float, float)
            Center of circle
    Return
    ------
        vertices
            Vertices with coordinates to draw the circle
    '''
    
    pi = 3.14
    angle = 0.0
    vertices = []
    
    for counter in range(n_vertices):
        angle += 2*pi/n_vertices
        x = center[0] + math.cos(angle) * radius
        y = center[1] + math.sin(angle) * radius
        vertices.append((x,y))
    
    return vertices

In [5]:
def setColor(loc_color, color):
    glUniform4f(loc_color, color[0], color[1], color[2], color[3])

In [6]:
def multiply_matrix(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

In [7]:
def updateCoordinates(coord_aux, mat):
    coord = []
    for x in range (len(coord_aux)):
        mult = np.dot(mat.reshape(4,4), np.array(coord_aux[x]+ (0,1)))
        coord.append((mult[0], mult[1]))
    return coord

In [8]:
def getCenter(coord):
    x = 0
    y = 0
    size = len(coord)
    for i in range(size):
        x += coord[i][0]
        y += coord[i][1]
    return x/size, y/size

In [9]:
# Colors's Normalized RGB
red    = (1.0  , 0    , 0    , 1.0)
blue   = (0    , 0    , 1.0  , 1.0)
yellow = (1.0  , 0.96 , 0    , 1.0)
purple = (0.349, 0.231, 0.427, 1.0)
orange = (1.0  , 0.658, 0    , 1.0)
green  = (0.282, 0.737, 0.184, 1.0)
black  = (0    , 0    , 0    , 1.0)
gray   = (0.278, 0.278, 0.278, 1.0)
dark_gray = (0.424, 0.424, 0.424, 1.0)
white  = (1.0  , 1.0  , 1.0  , 1.0)
brown = (0.172, 0.094, 0.027, 1.0)

In [10]:
# Matrizes Auxiliares
# WITCH
mat_witch_face = np.array([ 0.18,  0.,    0. ,  -0.02 , 0.  ,  0.19,  0.  ,  0.,    0. ,   0. ,   1.,    0.,
   0. ,   0.  ,  0. ,   1.  ], np.float32)
#CAT
mat_cat_body = np.array([       0.51, 0.,   0.,   0.,   
                                0.,   0.3,  0.,   0.,   
                                0.,   0.,   1.,   0.,   
                                0.,   0.,   0.,   1. ], np.float32)

mat_cat_head = np.array([0.31,    0.,      0.,     -0.0775,  
                         0.,      0.31,    0.,      0.0682,  
                         0.,      0.,      1.,      0.,      
                         0.,      0.,      0.,      1. ], np.float32)

mat_cat_left_eye = np.array([1.,     0.,     0.,    -0.099,  0.,     1.,     0.,     0.087,  0.,     0.,
   1.,     0.,     0.,     0.,     0.,     1. ], np.float32)
mat_cat_right_eye = np.array([1.,     0.,     0.,    -0.063,  0.,     1.,     0.,     0.086,  0.,     0.,
   1.,     0.,     0.,     0.,     0.,     1.], np.float32)

#CAULDRON
mat_lower_cauldron = np.array([       -0.63607824, -0.77162457,  0.0,          0.0,          
                                       0.77162457, -0.63607824, 0.0,          0.0,          
                                       0.0,          0.0,          1.0,          0.0,
                                       0.0,          0.0,          0.0,          1.], np.float32)
    
mat_upper_cauldron = np.array([        -3.1529682e+00,  1.2936424e-03,  0.0000000e+00, -2.4089683e-04,
                                       1.2664285e-01, -1.2744498e+00,  0.0000000e+00,  2.6241225e-01,
                                       0.0000000e+00,  0.0000000e+00,  2.0000000e+00,  0.0000000e+00,
                                       0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  2.0000000e+00], np.float32)

mat_bubble1 = np.array([1.,    0.,    0.,    0.12,  
                        0.,    1.,    0.,   -0.1,   
                        0.,    0.,    1.,    0.,
                        0.,    0.,    0.,    1.], np.float32)

mat_bubble2 = np.array([1.,   0.,   0.,   0.04, 
                        0.,   1.,   0.,   0.02, 
                        0.,   0.,   1.,   0.,   
                        0.,   0.,   0.,   1.,  ], np.float32)

mat_bubble3 = np.array([1.,    0.,    0.,   -0.06,  
                        0.,    1.,    0.,   -0.04,  
                        0.,    0.,    1.,    0.,
                        0.,    0.,    0.,    1.], np.float32)

mat_bubble4 = np.array([1.,    0.,    0.,   -0.12,  
                        0.,    1.,    0.,    0.03,  
                        0.,    0.,    1.,    0.,
                        0.,    0.,    0.,    1.], np.float32)

### BAT

mat_bat_body = np.array([-0.539533 ,  -0.0124742 ,  0.   ,       0.04   ,     0.02245356 ,-0.29974055,
   0.  ,        0.02    ,    0.    ,      0.    ,      1.  ,        0.,
   0.    ,      0.   ,       0.  ,        1.        ], np.float32)

mat_bat_head = np.array([0.18, 0. ,  0. ,  0.05, 0. , 0.18, 0.,   0.04, 0.,   0.  , 1. ,  0. ,  0. ,  0.,
  0. ,  1.  ], np.float32)

# ASSASSIN
mat_assassin_head = np.array([0.22, 0.,   0.,   0.53, 0.,   0.3,  0.,   0.24, 0.,   0.,   1.,   0.,   0.,   0.,
  0.,   1.  ], np.float32)


# Matrizes de montagem de cenario

mat_trans_witch = np.array([1. ,  0. ,  0. ,  0.49, 0. ,  1. ,  0.,   0.67, 0. ,  0.  , 1. ,  0.  , 0. ,  0.,
 0. ,  1.  ])

mat_trans_cat = np.array([ 1.,    0. ,   0.   ,-0.62,  0. ,   1.,    0.  , -0.44,  0. ,   0. ,   1.  ,  0.,
  0.  ,  0.  ,  0. ,   1.  ])

mat_trans_cauldron = np.array([ 1.,    0. ,   0. ,  -0.02,  0.,    1.,    0. ,  -0.32 , 0. ,   0.  ,  1.,    0.,
  0.  ,  0. ,   0. ,   1.  ])

mat_trans_bat = np.array([ 1.,    0. ,   0.  , -0.76,  0.,    1. ,   0. ,   0.17 , 0. ,   0. ,   1. ,   0.,
  0. ,   0.  ,  0.,    1.  ])

mat_trans_assassin = np.array([ 1.  ,  0.,    0.    ,0.24,  0.  ,  1.,    0. ,  -0.23 , 0.  ,  0. ,   1. ,  0.,
  0. ,   0.   , 0. ,   1.  ])
mat_trans_floor = np.array([ 1.,    0. ,   0.  ,  0. ,   0.  ,  1. ,   0. ,  -0.36,  0.  ,  0.,    1.  ,  0.,
  0.  ,  0.  ,  0.   , 1.  ])

In [11]:
# Coordinates
coord = []

### WITCH
# 0   BROOMSTICK
coord += [      (-0.1952286, -0.11131832),
                (-0.19363542,  -0.10144605),
                ( 0.15030092, -0.1670797),
                ( 0.1518941,  -0.15720743)  ]

# 4   BROOM
coord += [      (0.14051772, -0.19966869), 
                (0.15125949, -0.14063807),
                (0.24874052, -0.21936194),
                (0.25948229, -0.16033132)  ]

# 8   BODY
coord += [      (-0.01671812, -0.28686973), 
                (-0.04308998,  0.04207486),
                ( 0.04308998, -0.28207485),
                ( 0.01671812, 0.04686974)  ]

# 12  HAT 1
coord += [      (-0.1240334, -0.0139666), 
                (0.08178057, 0.02775396),
                ( -0.04 , 0.1 ),  ]



# 15  HAT 2
coord += [      (-0.07347928,  0.03998523), 
                (0.02580158, 0.05195645),
                ( -0.04    ,    0.18000001),  ]

# 18   FACE
coord_aux = getCircleCoordinates(32, 0.3)
coord += updateCoordinates(coord_aux, mat_witch_face)

### CAT 

# 50 Cat Body
coord_aux = getCircleCoordinates(32, 0.2) # 0-31
coord += updateCoordinates(coord_aux, mat_cat_body)

# 82  Cat Head
coord += updateCoordinates(coord_aux, mat_cat_head)

# 114 Cat Ears 
coord.append((-0.078,  0.112)) # Left: 64 - 66
coord.append((-0.098,  0.152))
coord.append((-0.118,  0.112))

coord.append((-0.038,  0.112)) # Right: 67 - 69
coord.append((-0.058,  0.152))
coord.append((-0.078,  0.112))

# 120 Cat Eyes
coord_aux = getCircleCoordinates(16, 0.01) 
coord += updateCoordinates(coord_aux, mat_cat_left_eye)  # 120-135
xle, yle = getCenter(coord[120:136])
coord += updateCoordinates(coord_aux, mat_cat_right_eye) # 136-151
xre, yre = getCenter(coord[136:152])



### CAULDRON

# 152 lower 
coord_aux = getCircleCoordinates(32, 0.2) # 0-31
coord += updateCoordinates(coord_aux, mat_lower_cauldron)

# 184 upper
coord += getCircleCoordinates(32, 0.1)

# 216 bubbles
coord += getCircleCoordinates(16, 0.02)

### BAT
# 232 bat body
coord_aux = getCircleCoordinates(32, 0.3)
coord += updateCoordinates(coord_aux, mat_bat_body)

# 264  bat head
coord_aux = getCircleCoordinates(32, 0.3)
coord += updateCoordinates(coord_aux, mat_bat_head)

#296 bat left ear
coord += [
            ( 0.04, 0.06),
            ( 0. ,  0.06),
            ( 0.02 ,0.14)
                            ]
#299 bat right ear
coord += [
            ( 0.105, 0.04),
            ( 0.055, 0.04),
            ( 0.08, 0.14)
                            ]

### ASSASSIN
xr = 0.48113062
yr = 0.03791015
#302 assassin's body

coord += [
    ( 0.47599997, -0.25),
    (0.58399997, -0.25),
    (0.47599997, 0.164),
    (0.58399997, 0.164) ]

#306 assasssin's head
coord_aux = getCircleCoordinates(32, 0.3)
coord += updateCoordinates(coord_aux, mat_assassin_head)

#338 assasssin's legs
coord += [
    (0.49099997, -0.49000001),
    (0.56899997, -0.49000001),
    (0.49099997, -0.23200001),
    (0.56899997, -0.23200001) ]

#342 assasssin's arms

coord += [
    (0.21886937, 0.10208986),
    (0.48113062, 0.03791015),
    (0.22600045, 0.13123),
    (0.4882617, 0.06705029) ]

#346 knife handle
coord += [
    (0.21125796, 0.03213932),
    (0.22874204, 0.02786068),
    (0.23978227, 0.14869988),
    (0.25726636, 0.14442124) ]

#350 knife blade
coord += [
    (0.21241307, 0.06482218),
    (0.18758693, -0.06482218),
    (0.26544941, 0.05466603) ]

#353 CHAO

coord += [
    (-2, 0),
    ( 2, 0),
    (-2,-2) ,
    ( 2,-2) ]


In [12]:
window, program, loc_color = initWindow("The Witch's Night", coord, key_event, mouse_event, 1000, 1000)

In [13]:
while not glfw.window_should_close(window):

    glfw.poll_events()
    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(0.349, 0.231, 0.427, 1.0)
    
    ### WITCH
    if(witch_spin == True):
        witch_theta -= 0.004
    
    if(witch_theta < -2* 3.14):
        witch_theta = 0
        witch_spin = False
        
    mat_witch_translation = np.array([      1.0, 0.0, 0.0, witch_tx, 
                                            0.0, 1.0, 0.0, 0.0, 
                                            0.0, 0.0, 1.0, 0.0, 
                                            0.0, 0.0, 0.0, 1.0], np.float32)
    
    mat_witch_rotation = np.array([         math.cos(witch_theta), -math.sin(witch_theta), 0.0, 0.0, 
                                            math.sin(witch_theta),  math.cos(witch_theta), 0.0, 0.0, 
                                            0.0,              0.0,             1.0, 0.0, 
                                            0.0,              0.0,             0.0, 1.0], np.float32)
    
    ### CAT
    if(cat_sleeping):
        if(s_eyes >= 0.0):
            s_eyes -= 0.001
    else:
        if(s_eyes <= 1.0):
            s_eyes += 0.001
    
    
    mat_left_eyes = np.array([        1.0, 0.0,    0.0, 0.0, 
                                      0.0, s_eyes, 0.0, yle*(1-s_eyes), 
                                      0.0, 0.0,    1.0, 0.0, 
                                      0.0, 0.0,    0.0, 1.0], np.float32)
    
    mat_right_eyes = np.array([       1.0, 0.0,    0.0, 0.0, 
                                      0.0, s_eyes, 0.0, yre*(1-s_eyes), 
                                      0.0, 0.0,    1.0, 0.0, 
                                      0.0, 0.0,    0.0, 1.0], np.float32)
    ### CAULDRON
    
    t_bubble += 0.00005
    if t_bubble > 0.3:
        t_bubble = 0.0
        
    mat_translation = np.array([    1.0, 0.0, 0.0, 0.0, 
                                    0.0, 1.0, 0.0, t_bubble, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)

    

    mat_i = np.array([              1.0, 0.0, 0.0, 0.0, 
                                    0.0, 1.0, 0.0, 0.0, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32) 
    
    ### BAT
    if(bat_scale == True):
        s_x += 0.02
        s_y += 0.02
    
    if(s_x > 20):
#         s_x = 1
#         s_y = 1
        bat_scale = False
        
    if(bat_scale == False and s_x > 1):
        s_x -= 0.02
        s_y -= 0.02
        
    mat_scaling = np.array([          s_x, 0.0, 0.0, 0.0, 
                                      0.0, s_y, 0.0, 0.0, 
                                      0.0, 0.0, 1.0, 0.0, 
                                      0.0, 0.0, 0.0, 1.0], np.float32)

    ### ASSASSIN
    mat_refrot = np.array([         math.cos(arm_rotation), -math.sin(arm_rotation), 0.0, xr - xr*math.cos(arm_rotation) + yr*math.sin(arm_rotation), 
                                    math.sin(arm_rotation),  math.cos(arm_rotation), 0.0, yr - yr*math.cos(arm_rotation) - xr*math.sin(arm_rotation), 
                                    0.0,              0.0,             1.0, 0.0, 
                                    0.0,              0.0,             0.0, 1.0], np.float32)
    
    if (arm_rotation > 2 * np.pi / 15.0 ):
        arm_rotation = 2 * np.pi / 15.0
    if (arm_rotation < - 2 * np.pi / 15.0 ):
        arm_rotation = -2 * np.pi / 15.0
    
    loc = glGetUniformLocation(program, "mat_transformation")

    # DRAW FLOOR
    mat_transformation =  mat_trans_floor
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, brown)
    draw_squares(353)

    ### DRAW WITCH
    ###########################################################################################################
    
    #   BROOMSTICK 
    if(witch_spin == True):
        mat_transformation = multiply_matrix(mat_witch_translation, mat_witch_rotation)
        mat_transformation = multiply_matrix(mat_trans_witch, mat_transformation)
    else:
        mat_transformation = mat_witch_translation
        mat_transformation = multiply_matrix(mat_trans_witch, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, black)
    draw_squares(0) 
    
    #   BROOM
    if(witch_spin == True):
        mat_transformation = multiply_matrix(mat_witch_translation, mat_witch_rotation)
        mat_transformation = multiply_matrix(mat_trans_witch, mat_transformation)
    else:
        mat_transformation = mat_witch_translation 
        mat_transformation = multiply_matrix(mat_trans_witch, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, black)
    draw_squares(4) 
    
    #   BODY
    if(witch_spin == True):
        mat_transformation = multiply_matrix(mat_witch_translation, mat_witch_rotation)
        mat_transformation = multiply_matrix(mat_trans_witch, mat_transformation)
    else:
        mat_transformation = mat_witch_translation
        mat_transformation = multiply_matrix(mat_trans_witch, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, black)
    draw_squares(8) 
    
    #   HAT 1
    if(witch_spin == True):
        mat_transformation = multiply_matrix(mat_witch_translation, mat_witch_rotation)
        mat_transformation = multiply_matrix(mat_trans_witch, mat_transformation)
    else:
        mat_transformation = mat_witch_translation
        mat_transformation = multiply_matrix(mat_trans_witch, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, black)
    draw_triangles(12) 
    
    #   HAT 2
    if(witch_spin == True):
        mat_transformation = multiply_matrix(mat_witch_translation, mat_witch_rotation)
        mat_transformation = multiply_matrix(mat_trans_witch, mat_transformation)
    else:
        mat_transformation = mat_witch_translation
        mat_transformation = multiply_matrix(mat_trans_witch, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, black)
    draw_triangles(15) 
    
    #   FACE
    if(witch_spin == True):
        mat_transformation = multiply_matrix(mat_witch_translation, mat_witch_rotation)
        mat_transformation = multiply_matrix(mat_trans_witch, mat_transformation)
    else:
        mat_transformation = mat_witch_translation
        mat_transformation = multiply_matrix(mat_trans_witch, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, black)
    draw_circles(18, 32) 
    
    ### DRAW CAT
    #################################################################################################
    
    # Draw Cat Body    
    mat_transformation = mat_i
    mat_transformation = multiply_matrix(mat_trans_cat, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, black)
    draw_circles(50, 32)
    
    # Draw Cat Head    
    mat_transformation = mat_i
    mat_transformation = multiply_matrix(mat_trans_cat, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, black)
    draw_circles(82, 32)
    
    
    # Draw Cat Ears
    mat_transformation = mat_i
    mat_transformation = multiply_matrix(mat_trans_cat, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, black)
    draw_triangles(114, 6)
    
    # Draw Cat Left Eyes
    mat_transformation = mat_left_eyes
    mat_transformation = multiply_matrix(mat_trans_cat, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, yellow)
    draw_circles(120, 16)
    
    # Draw Cat Right Eyes
    mat_transformation = mat_right_eyes
    mat_transformation = multiply_matrix(mat_trans_cat, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, yellow)
    draw_circles(136, 16)
    
    ### DRAW CAULDRON
    ################################################################################################
    
    mat_transformation = multiply_matrix(mat_bubble1, mat_translation)
    mat_transformation = multiply_matrix(mat_trans_cauldron, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, green)
    draw_circles(216, 16)
    
    mat_transformation = multiply_matrix(mat_bubble2, mat_translation)
    mat_transformation = multiply_matrix(mat_trans_cauldron, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, green)
    draw_circles(216, 16)
    
    mat_transformation = multiply_matrix(mat_bubble3, mat_translation)
    mat_transformation = multiply_matrix(mat_trans_cauldron, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, green)
    draw_circles(216, 16)
    
    mat_transformation = multiply_matrix(mat_bubble4, mat_translation)
    mat_transformation = multiply_matrix(mat_trans_cauldron, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, green)
    draw_circles(216, 16)
    
    # Draw Lower Cauldron
    mat_transformation = mat_i
    mat_transformation = multiply_matrix(mat_trans_cauldron, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, gray)
    draw_circles(152, 24)
    
    # Draw Upper Cauldron
    mat_transformation = mat_upper_cauldron
    mat_transformation = multiply_matrix(mat_trans_cauldron, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, green)
    draw_circles(184, 16)
    
    ### DRAW BAT
    ################################################################################################
    
    mat_transformation = multiply_matrix(mat_i, mat_scaling)
    mat_transformation = multiply_matrix(mat_trans_bat, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, black)
    draw_circles(232, 16) 
    
    mat_transformation = multiply_matrix(mat_i, mat_scaling)
    mat_transformation = multiply_matrix(mat_trans_bat, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, black)
    draw_circles(264, 32) 
    
    mat_transformation = multiply_matrix(mat_i, mat_scaling)
    mat_transformation = multiply_matrix(mat_trans_bat, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, black)
    draw_triangles(296)  
    
    mat_transformation = multiply_matrix(mat_i, mat_scaling)
    mat_transformation = multiply_matrix(mat_trans_bat, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, black)
    draw_triangles(299)  
    # print(mat_transformation)
    # mat_transformation = multiply_matrix(multiply_matrix(mat_translation, mat_rotation), mat_scaling)
    
    ### DRAW ASSASSIN
    ################################################################################################
    
    #   Body
    mat_transformation = mat_i
    mat_transformation = multiply_matrix(mat_trans_assassin, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, black)
    draw_squares(302) 
    
    #   Head
    mat_transformation = mat_i
    mat_transformation = multiply_matrix(mat_trans_assassin, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, black)
    draw_circles(306, 32) 
    
    #   Legs
    mat_transformation =  mat_i
    mat_transformation = multiply_matrix(mat_trans_assassin, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, black)
    draw_squares(338)  
    
    #   Arm
    mat_transformation =  mat_refrot
    mat_transformation = multiply_matrix(mat_trans_assassin, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, black)
    draw_squares(342)  
    
    #   Knife Handle
    mat_transformation =  mat_refrot
    mat_transformation = multiply_matrix(mat_trans_assassin, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, black)
    draw_squares(346) 
    
    #   Knife Blade
    mat_transformation =   mat_refrot
    mat_transformation = multiply_matrix(mat_trans_assassin, mat_transformation)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
    setColor(loc_color, black)
    draw_triangles(350)  
    
    
    glfw.swap_buffers(window)

glfw.terminate()